<a href="https://colab.research.google.com/github/roldanpau/Titanic/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tensor flow development

In [21]:
# mlp for binary classification
import pandas as pd
import numpy as np
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', None) # Display all columns of dataframes

In [22]:
# PARAMETERS
val_size = 100    # size of validation set

In [23]:
# Download dataset
TITANIC_TRAIN = "https://raw.githubusercontent.com/borjagomez/TF-Titanic/master/data/train.csv"
TITANIC_TEST = "https://raw.githubusercontent.com/borjagomez/TF-Titanic/master/data/test.csv"

train_data = pd.read_csv(TITANIC_TRAIN)
test_data = pd.read_csv(TITANIC_TEST)

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
y = train_data["Survived"]

# Use only these features, converting categorical variables (such as "Sex" and 
# "Embarked") into indicator (or "dummy") variables
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

# To scale X and X_test homogeneously, we concat them in a single dataframe, X2.
X2 = pd.concat([X, X_test])

# Show which columns have unknown values
print(X2.isna().sum())

# We have NaN values in Age and Fare. 
# Substitute missing "Fare" values by the average.
X2["Fare"] = X2.Fare.fillna(X2.Fare.mean())

Pclass          0
Age           263
SibSp           0
Parch           0
Fare            1
Sex_female      0
Sex_male        0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64


In [25]:
# Predict missing "Age" values
cols = ["Pclass", "SibSp", "Parch", "Age"]
df = X2[cols]

age_test_data = df[df["Age"].isnull()]
age_data = df.dropna(inplace=False)

age_y_train = age_data["Age"]
age_X_train = age_data.drop("Age", axis=1)
age_y_test = age_test_data["Age"]
age_X_test = age_test_data.drop("Age", axis=1)

age_model = LinearRegression()
age_model.fit(age_X_train, age_y_train)

age_y_pred = age_model.predict(age_X_test)

#replace the missing values with predicted values
X2.loc[X2.Age.isnull(), 'Age'] = age_y_pred

print(X2.isna().sum())

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64


In [26]:
scaler = MinMaxScaler()
X2 = pd.DataFrame(scaler.fit_transform(X2), columns=X2.columns)
#print(X2.head())

print(X.shape[0])

X = X2.iloc[:X.shape[0], :]
X_test = X2.iloc[X.shape[0]:, :]
#print(X.head())
#print(X.tail())
#print(X_test.head())
#print(X_test.tail())

891


In [27]:
print(X)
print(X_test)
print(y)

     Pclass       Age  SibSp     Parch      Fare  Sex_female  Sex_male  \
0       1.0  0.283600  0.125  0.000000  0.014151         0.0       1.0   
1       0.0  0.481228  0.125  0.000000  0.139136         1.0       0.0   
2       1.0  0.333007  0.000  0.000000  0.015469         1.0       0.0   
3       0.0  0.444173  0.125  0.000000  0.103644         1.0       0.0   
4       1.0  0.444173  0.000  0.000000  0.015713         0.0       1.0   
..      ...       ...    ...       ...       ...         ...       ...   
886     0.5  0.345359  0.000  0.000000  0.025374         0.0       1.0   
887     0.0  0.246545  0.000  0.000000  0.058556         1.0       0.0   
888     1.0  0.271691  0.125  0.222222  0.045771         1.0       0.0   
889     0.0  0.333007  0.000  0.000000  0.058556         0.0       1.0   
890     1.0  0.407117  0.000  0.000000  0.015127         0.0       1.0   

     Embarked_C  Embarked_Q  Embarked_S  
0           0.0         0.0         1.0  
1           1.0         0.0

To evaluate how well the trained model performs on unseen data, we further split the dataset into separate training and test datasets.

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  test_size=0.125, 
                                                  random_state=1, 
                                                  stratify=y)
# Reserve val_size samples for validation
# X_val = X[-val_size:]
# y_val = y[-val_size:]
# X_train = X[:-val_size]
# y_train = y[:-val_size]

print(X_train)
print(y_train)

     Pclass       Age  SibSp     Parch      Fare  Sex_female  Sex_male  \
35      0.0  0.530635  0.125  0.000000  0.101497         0.0       1.0   
787     1.0  0.110676  0.500  0.111111  0.056848         0.0       1.0   
164     1.0  0.024214  0.500  0.111111  0.077465         0.0       1.0   
780     1.0  0.172435  0.000  0.000000  0.014110         1.0       0.0   
239     0.5  0.419469  0.000  0.000000  0.023959         0.0       1.0   
..      ...       ...    ...       ...       ...         ...       ...   
338     1.0  0.567690  0.000  0.000000  0.015713         0.0       1.0   
265     0.5  0.456524  0.000  0.000000  0.020495         0.0       1.0   
546     0.5  0.246545  0.125  0.000000  0.050749         1.0       0.0   
413     0.5  0.424118  0.000  0.000000  0.000000         0.0       1.0   
464     1.0  0.340018  0.000  0.000000  0.015713         0.0       1.0   

     Embarked_C  Embarked_Q  Embarked_S  
35          0.0         0.0         1.0  
787         0.0         1.0

In [29]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the validation set
y_pred = np.rint(regr.predict(X_val)).astype(int)

#print(y_pred)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_val, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_val, y_pred))
# The accuracy score (fraction of correctly classified samples):
print('Accuracy score: %.2f'
      % accuracy_score(y_val, y_pred))

# Plot outputs
#plt.scatter(X_val, y_val,  color='black')
#plt.plot(X_val, y_pred, color='blue', linewidth=3)

#plt.xticks(())
#plt.yticks(())

#plt.show()

Coefficients: 
 [-0.36286697 -0.55631929 -0.51440088 -0.16732749  0.15946178  0.25577758
 -0.25577758 -0.10177919 -0.07499245 -0.15683067]
Mean squared error: 0.24
Coefficient of determination: -0.02
Accuracy score: 0.76


Evaluate the model on the test data and 
save predictions to file.

In [30]:
# Generate predictions for the passengers in test.csv
predictions = np.rint(regr.predict(X_test).flatten()).astype(int)

#print(predictions)
#print(test_data.PassengerId)

# Save these new predictions in a CSV file my_submission.csv.
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 
                       'Survived': predictions})
output.to_csv('my_submission_LR.csv', index=False)

print("Your submission was successfully saved!")

Your submission was successfully saved!
